In [1]:
import pandas as pd
import numpy as np
import requests
import duckdb
from shapely.geometry import Point
import sys
import sqlalchemy
import IPython
from dotenv import load_dotenv
import json
from urllib.request import urlopen
import geopandas as gpd
import dataframe_image as dfi

In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [4]:
conn = duckdb.connect('../data/bicimad.db')

## SQLite DataBase

In [5]:
database = conn.sql('SELECT * from bicimad_stations')
database

┌───────┬──────────────────────┬───────┬───┬────────────┬────────────────────┬───────────────┬──────────────────────┐
│  id   │         name         │ light │ … │ free_bases │ reservations_count │ geometry.type │ geometry.coordinates │
│ int64 │       varchar        │ int64 │   │   int64    │       int64        │    varchar    │       varchar        │
├───────┼──────────────────────┼───────┼───┼────────────┼────────────────────┼───────────────┼──────────────────────┤
│     1 │ 1a - Puerta del So…  │     3 │ … │          0 │                  0 │ Point         │ [-3.7018341, 40.41…  │
│     2 │ 1b - Puerta del So…  │     3 │ … │          0 │                  0 │ Point         │ [-3.70160293806045…  │
│     3 │ 2 - Miguel Moya      │     3 │ … │          0 │                  0 │ Point         │ [-3.7058415, 40.42…  │
│     4 │ 3 - Plaza Conde Su…  │     2 │ … │          9 │                  0 │ Point         │ [-3.7069171, 40.43…  │
│     5 │ 4 - Malasaña         │     1 │ … │          1 

In [6]:
#bicimad_filtered = conn.sql("SELECT id, address, 'geometry.coordinates' as 'coordinates' FROM bicimad_stations")
#bicimad_filtered

In [7]:
bicimad = database.df()
bicimad

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]"
260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]"
261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]"
262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]"


In [8]:
bicimad["Type of Place"] = "Principales espacios de deporte"
bicimad

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,Type of Place
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",Principales espacios de deporte
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]",Principales espacios de deporte
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]",Principales espacios de deporte
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]",Principales espacios de deporte
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]",Principales espacios de deporte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]",Principales espacios de deporte
260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]",Principales espacios de deporte
261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]",Principales espacios de deporte
262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]",Principales espacios de deporte


In [9]:
def bicimad_longitud(geo):
    coordinates = str(geo).replace("[","").replace("]","").split(",")
    longitude = coordinates[0].strip()
    return longitude

In [10]:
bicimad["Longitude"] = bicimad["geometry.coordinates"].apply(bicimad_longitud)
bicimad.head()

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,Type of Place,Longitude
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",Principales espacios de deporte,-3.7018341
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]",Principales espacios de deporte,-3.701602938060457
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]",Principales espacios de deporte,-3.7058415
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]",Principales espacios de deporte,-3.7069171
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]",Principales espacios de deporte,-3.7025875


In [11]:
def bicimad_latitude(geo):
    coordinates = str(geo).replace("[","").replace("]","").split(",")
    latitude = coordinates[1].strip()
    return latitude

In [12]:
bicimad["Latitude"] = bicimad["geometry.coordinates"].apply(bicimad_latitude)
bicimad.head()

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,Type of Place,Longitude,Latitude
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",Principales espacios de deporte,-3.7018341,40.4172137
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]",Principales espacios de deporte,-3.701602938060457,40.41731271011562
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]",Principales espacios de deporte,-3.7058415,40.4205886
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]",Principales espacios de deporte,-3.7069171,40.4302937
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]",Principales espacios de deporte,-3.7025875,40.4285524


In [13]:
bicimad_final = bicimad[['name', 'address', 'Type of Place', 'Longitude', 'Latitude']]
bicimad_final

,name,address,Type of Place,Longitude,Latitude
0,1a - Puerta del Sol A,Puerta del Sol nº 1,Principales espacios de deporte,-3.7018341,40.4172137
1,1b - Puerta del Sol B,Puerta del Sol nº 1,Principales espacios de deporte,-3.701602938060457,40.41731271011562
2,2 - Miguel Moya,Calle Miguel Moya nº 1,Principales espacios de deporte,-3.7058415,40.4205886
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,Principales espacios de deporte,-3.7069171,40.4302937
4,4 - Malasaña,Calle Manuela Malasaña nº 5,Principales espacios de deporte,-3.7025875,40.4285524
...,...,...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,Principales espacios de deporte,-3.72997,40.43896
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,Principales espacios de deporte,-3.72699,40.44375
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,Principales espacios de deporte,-3.72693,40.44342
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,Principales espacios de deporte,-3.7272945,40.4483322


## API REST

In [14]:
url = 'https://datos.madrid.es/egob/catalogo/212808-0-espacio-deporte.json' 
headers = {'Accept': 'application/json'}
espacios_json = requests.get('https://datos.madrid.es/egob/catalogo/212808-0-espacio-deporte.json', headers=headers).json()
espacios_json

In [15]:
espacios_fd = pd.json_normalize(espacios_json, ['@graph'])
espacios_fd

,@id,@type,id,title,relation,address.district.@id,address.area.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.accesibility,organization.schedule,organization.services,organization.organization-name
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,181447,Campo de Golf del Centro Nacional de Golf,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28049,CALLE ARROYO DEL MONTE 5,40.485508,-3.735279,"Bus: 67, 82.",3,,,Campo de Golf del Centro Nacional de Golf
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,4670,Campo de Golf del Club de Golf Olivar de la Hi...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28042,CALLE VIA DE DUBLIN SIN NÚMERO,40.468966,-3.609367,Bus: 112.,3,,,Campo de Golf del Club de Golf Olivar de la Hi...
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,7381184,Centro Comercial Dreams - Palacio Municipal de...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28043,CALLE SILVANO 77,40.462872,-3.635776,"Metro: Canillas (línea 4). Bus: 120, 112, 1...",3,,,Centro Comercial Dreams - Palacio Municipal de...
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,7081762,Centro de Alto Rendimiento del Consejo Superio...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28040,CALLE MARTIN FIERRO 5,40.436812,-3.731957,"Bus: 46, U, 160, 161. Bicimad: Estación 257...",3,,,Centro de Alto Rendimiento del Consejo Superio...
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,4844,Centro de Natación Mundial 86 (M - 86),http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28007,CALLE JOSE MARTINEZ DE VELASCO 3,40.415523,-3.667197,Metro: Sainz de Baranda (líneas 6 y 9) Bus:...,1,Apertura de la instalación: De lunes a viern...,Piscina de 8 calles al aire libre (con cubiert...,Centro de Natación Mundial 86 (M - 86)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,8192515,Parque Deportivo Puerta de Hierro,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28023,CARRETERA DE LA CORUÑA kilómetro 7,40.463352,-3.749646,"Bus: 83, 133.",1,Apertura de la instalación: De lunes a domingo...,,Parque Deportivo Puerta de Hierro
598,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10967962,Parque Lineal UZI 0.06 Arroyofresno,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28049,CALLE MARIA DE MAEZTU 1,40.487147,-3.732863,"Metro: Antonio Machado.Bus: 42 , 49 , 126 , ...",0,,Pista de baloncesto.,Parque Lineal UZI 0.06 Arroyofresno
599,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10967954,Pista de Patinaje Paco de Lucía,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,NaN,MADRID,,AUDITORIO PACO DE LUCIA,40.385278,-3.764155,"Metro: Antonio Machado.Bus: 42 , 49 , 126 , ...",0,,Pista de pa

In [16]:
espacios_final = espacios_fd[['title', 'location.latitude', 'location.longitude', 'address.street-address']]
espacios_final

,title,location.latitude,location.longitude,address.street-address
0,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5
1,Campo de Golf del Club de Golf Olivar de la Hi...,40.468966,-3.609367,CALLE VIA DE DUBLIN SIN NÚMERO
2,Centro Comercial Dreams - Palacio Municipal de...,40.462872,-3.635776,CALLE SILVANO 77
3,Centro de Alto Rendimiento del Consejo Superio...,40.436812,-3.731957,CALLE MARTIN FIERRO 5
4,Centro de Natación Mundial 86 (M - 86),40.415523,-3.667197,CALLE JOSE MARTINEZ DE VELASCO 3
...,...,...,...,...
597,Parque Deportivo Puerta de Hierro,40.463352,-3.749646,CARRETERA DE LA CORUÑA kilómetro 7
598,Parque Lineal UZI 0.06 Arroyofresno,40.487147,-3.732863,CALLE MARIA DE MAEZTU 1
599,Pista de Patinaje Paco de Lucía,40.385278,-3.764155,AUDITORIO PACO DE LUCIA
600,Pista de petanca Centro de Mayores La Vaguada,40.479033,-3.708264,AVENIDA MONFORTE DE LEMOS 36


In [17]:
espacios_json.keys()

dict_keys(['@context', '@graph'])

In [18]:
espacios_json['@context']

{'c': 'http://www.w3.org/2002/12/cal#',
 'dcterms': 'http://purl.org/dc/terms/',
 'geo': 'http://www.w3.org/2003/01/geo/wgs84_pos#',
 'loc': 'http://purl.org/ctic/infraestructuras/localizacion#',
 'org': 'http://purl.org/ctic/infraestructuras/organizacion#',
 'vcard': 'http://www.w3.org/2006/vcard/ns#',
 'schema': 'https://schema.org/',
 'title': 'vcard:fn',
 'id': 'dcterms:identifier',
 'relation': 'dcterms:relation',
 'references': 'dcterms:references',
 'address': 'vcard:adr',
 'area': 'loc:barrio',
 'district': 'loc:distrito',
 'locality': 'vcard:locality',
 'postal-code': 'vcard:postal-code',
 'street-address': 'vcard:street-address',
 'location': 'vcard:geo',
 'latitude': 'geo:lat',
 'longitude': 'geo:long',
 'organization': 'vcard:org',
 'organization-desc': 'dcterms:description',
 'accesibility': 'org:accesibilidad',
 'services': 'org:servicios',
 'schedule': 'org:horario',
 'organization-name': 'vcard:organization-name',
 'description': 'c:summary',
 'link': 'c:url',
 'uid': '

In [19]:
espacios_json['@graph']

[{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/181447-campo-golf-centro-nacional-golf.json',
  '@type': 'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosEspaciosDeporte/CamposGolf',
  'id': '181447',
  'title': 'Campo de Golf del Centro Nacional de Golf',
  'relation': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=bfa48ab43d6bb410VgnVCM100000171f5a0aRCRD&vgnextoid=198d38d60b41c010VgnVCM2000000c205a0aRCRD',
  'address': {'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Fuencarral-ElPardo'},
   'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Fuencarral-ElPardo/Barrio/Fuentelarreina'},
   'locality': 'MADRID',
   'postal-code': '28049',
   'street-address': 'CALLE ARROYO DEL MONTE 5'},
  'location': {'latitude': 40.4855081729451, 'longitude': -3.735278975476509},
  'organization': {'organization-desc': '\xa0\xa0Bus: 67, 82.',
   'accesibility': '3',


In [20]:
#espacios_deporte = pd.read_csv("../data/espacio-deporte.csv", sep=';')
#espacios_deporte

## Tabla final

In [21]:
tabla_final = espacios_final.assign(key=1).merge(bicimad_final.assign(key=1), how='outer', on = 'key').reset_index(drop=True)
tabla_final

,title,location.latitude,location.longitude,address.street-address,key,name,address,Type of Place,Longitude,Latitude
0,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,1a - Puerta del Sol A,Puerta del Sol nº 1,Principales espacios de deporte,-3.7018341,40.4172137
1,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,1b - Puerta del Sol B,Puerta del Sol nº 1,Principales espacios de deporte,-3.701602938060457,40.41731271011562
2,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,2 - Miguel Moya,Calle Miguel Moya nº 1,Principales espacios de deporte,-3.7058415,40.4205886
3,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,Principales espacios de deporte,-3.7069171,40.4302937
4,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,4 - Malasaña,Calle Manuela Malasaña nº 5,Principales espacios de deporte,-3.7025875,40.4285524
...,...,...,...,...,...,...,...,...,...,...
158923,WiZink Center,40.423273,-3.672101,CALLE JORGE JUAN 99,1,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,Principales espacios de deporte,-3.72997,40.43896
158924,WiZink Center,40.423273,-3.672101,CALLE JORGE JUAN 99,1,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,Principales espacios de deporte,-3.72699,40.44375
158925,WiZink Center,40.423273,-3.672101,CALLE JORGE JUAN 99,1,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,Principales espacios de deporte,-3.72693,40.44342
158926,WiZink Center,40.423273,-3.672101,CALLE JORGE JUAN 99,1,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,Principales espacios de deporte,-3.7272945,40.4483322


In [22]:
def distance_meters(Latitude1, Longitude1, Latitude, Longitude):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(Latitude1, Longitude1)
    finish = to_mercator(Latitude, Longitude)
    return start.distance(finish)

In [36]:
def to_mercator(lat, long):
    #transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

In [37]:
print(type(tabla_final['Latitude1'][0]))
print(type(tabla_final['Longitude1'][0]))
print(type(tabla_final['Latitude'][0]))
print(type(tabla_final['Longitude'][0]))

<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [38]:
tabla_final['Longitude'] = tabla_final['Longitude'].astype(float)
print(type(tabla_final['Longitude'][0]))

<class 'numpy.float64'>


In [39]:
tabla_final['Latitude'] = tabla_final['Latitude'].astype(float)
print(type(tabla_final['Latitude'][0]))

<class 'numpy.float64'>


In [40]:
tabla_final.rename(columns = {'location.latitude':'Latitude1', 'location.longitude':'Longitude1'}, inplace = True)
tabla_final

,Place of interest,Latitude1,Longitude1,Place Address,key,BiciMAD Station,Station Location,Type of Place,Longitude,Latitude
0,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,1a - Puerta del Sol A,Puerta del Sol nº 1,Principales espacios de deporte,-3.701834,40.417214
1,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,1b - Puerta del Sol B,Puerta del Sol nº 1,Principales espacios de deporte,-3.701603,40.417313
2,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,2 - Miguel Moya,Calle Miguel Moya nº 1,Principales espacios de deporte,-3.705842,40.420589
3,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,Principales espacios de deporte,-3.706917,40.430294
4,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,4 - Malasaña,Calle Manuela Malasaña nº 5,Principales espacios de deporte,-3.702587,40.428552
...,...,...,...,...,...,...,...,...,...,...
158923,WiZink Center,40.423273,-3.672101,CALLE JORGE JUAN 99,1,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,Principales espacios de deporte,-3.729970,40.438960
158924,WiZink Center,40.423273,-3.672101,CALLE JORGE JUAN 99,1,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,Principales espacios de deporte,-3.726990,40.443750
158925,WiZink Center,40.423273,-3.672101,CALLE JORGE JUAN 99,1,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,Principales espacios de deporte,-3.726930,40.443420
158926,WiZink Center,40.423273,-3.672101,CALLE JORGE JUAN 99,1,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,Principales espacios de deporte,-3.727295,40.448332


In [41]:
tabla_final.columns

Index(['Place of interest', 'Latitude1', 'Longitude1', 'Place Address', 'key',
       'BiciMAD Station', 'Station Location', 'Type of Place', 'Longitude',
       'Latitude'],
      dtype='object')

In [42]:
%time
tabla_final['Distance'] = tabla_final.apply(lambda x: distance_meters(x["Latitude1"], x["Longitude1"], x["Latitude"], x["Longitude"]), axis=1)

CPU times: user 5 µs, sys: 3 µs, total: 8 µs
Wall time: 14.8 µs


In [43]:
tabla_final

,Place of interest,Latitude1,Longitude1,Place Address,key,BiciMAD Station,Station Location,Type of Place,Longitude,Latitude,Distance
0,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,1a - Puerta del Sol A,Puerta del Sol nº 1,Principales espacios de deporte,-3.701834,40.417214,8468.640459
1,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,1b - Puerta del Sol B,Puerta del Sol nº 1,Principales espacios de deporte,-3.701603,40.417313,8470.152744
2,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,2 - Miguel Moya,Calle Miguel Moya nº 1,Principales espacios de deporte,-3.705842,40.420589,7937.930753
3,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,Principales espacios de deporte,-3.706917,40.430294,6912.964085
4,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,4 - Malasaña,Calle Manuela Malasaña nº 5,Principales espacios de deporte,-3.702587,40.428552,7314.298725
...,...,...,...,...,...,...,...,...,...,...,...
158923,WiZink Center,40.423273,-3.672101,CALLE JORGE JUAN 99,1,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,Principales espacios de deporte,-3.729970,40.438960,6687.479181
158924,WiZink Center,40.423273,-3.672101,CALLE JORGE JUAN 99,1,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,Principales espacios de deporte,-3.726990,40.443750,6533.565460
158925,WiZink Center,40.423273,-3.672101,CALLE JORGE JUAN 99,1,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,Principales espacios de deporte,-3.726930,40.443420,6514.555094
158926,WiZink Center,40.423273,-3.672101,CALLE JORGE JUAN 99,1,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,Principales espacios de deporte,-3.727295,40.448332,6759.459954


In [46]:
print(type(tabla_final['Distance'][0]))

<class 'numpy.float64'>


In [47]:
tabla_final.head(5)

,Place of interest,Latitude1,Longitude1,Place Address,key,BiciMAD Station,Station Location,Type of Place,Longitude,Latitude,Distance
0,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,1a - Puerta del Sol A,Puerta del Sol nº 1,Principales espacios de deporte,-3.701834,40.417214,8468.640459
1,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,1b - Puerta del Sol B,Puerta del Sol nº 1,Principales espacios de deporte,-3.701603,40.417313,8470.152744
2,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,2 - Miguel Moya,Calle Miguel Moya nº 1,Principales espacios de deporte,-3.705842,40.420589,7937.930753
3,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,Principales espacios de deporte,-3.706917,40.430294,6912.964085
4,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,4 - Malasaña,Calle Manuela Malasaña nº 5,Principales espacios de deporte,-3.702587,40.428552,7314.298725


## Cambio nombre de columnas

In [48]:
tabla_final.rename(columns = {'title':'Place of interest', 'address.street-address':'Place Address', 'name':'BiciMAD Station', 
                   'address':'Station Location'}, inplace = True)
tabla_final.head(5)

,Place of interest,Latitude1,Longitude1,Place Address,key,BiciMAD Station,Station Location,Type of Place,Longitude,Latitude,Distance
0,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,1a - Puerta del Sol A,Puerta del Sol nº 1,Principales espacios de deporte,-3.701834,40.417214,8468.640459
1,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,1b - Puerta del Sol B,Puerta del Sol nº 1,Principales espacios de deporte,-3.701603,40.417313,8470.152744
2,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,2 - Miguel Moya,Calle Miguel Moya nº 1,Principales espacios de deporte,-3.705842,40.420589,7937.930753
3,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,Principales espacios de deporte,-3.706917,40.430294,6912.964085
4,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,4 - Malasaña,Calle Manuela Malasaña nº 5,Principales espacios de deporte,-3.702587,40.428552,7314.298725


In [49]:
tabla_final.head(5)

,Place of interest,Latitude1,Longitude1,Place Address,key,BiciMAD Station,Station Location,Type of Place,Longitude,Latitude,Distance
0,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,1a - Puerta del Sol A,Puerta del Sol nº 1,Principales espacios de deporte,-3.701834,40.417214,8468.640459
1,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,1b - Puerta del Sol B,Puerta del Sol nº 1,Principales espacios de deporte,-3.701603,40.417313,8470.152744
2,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,2 - Miguel Moya,Calle Miguel Moya nº 1,Principales espacios de deporte,-3.705842,40.420589,7937.930753
3,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,Principales espacios de deporte,-3.706917,40.430294,6912.964085
4,Campo de Golf del Centro Nacional de Golf,40.485508,-3.735279,CALLE ARROYO DEL MONTE 5,1,4 - Malasaña,Calle Manuela Malasaña nº 5,Principales espacios de deporte,-3.702587,40.428552,7314.298725


## Cambio orden de columnas

In [50]:
tabla_final = tabla_final[['Place of interest', 'Type of Place', 'Place Address', 'BiciMAD Station', 'Station Location', 'Distance']]
tabla_final.head(5)

,Place of interest,Type of Place,Place Address,BiciMAD Station,Station Location,Distance
0,Campo de Golf del Centro Nacional de Golf,Principales espacios de deporte,CALLE ARROYO DEL MONTE 5,1a - Puerta del Sol A,Puerta del Sol nº 1,8468.640459
1,Campo de Golf del Centro Nacional de Golf,Principales espacios de deporte,CALLE ARROYO DEL MONTE 5,1b - Puerta del Sol B,Puerta del Sol nº 1,8470.152744
2,Campo de Golf del Centro Nacional de Golf,Principales espacios de deporte,CALLE ARROYO DEL MONTE 5,2 - Miguel Moya,Calle Miguel Moya nº 1,7937.930753
3,Campo de Golf del Centro Nacional de Golf,Principales espacios de deporte,CALLE ARROYO DEL MONTE 5,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,6912.964085
4,Campo de Golf del Centro Nacional de Golf,Principales espacios de deporte,CALLE ARROYO DEL MONTE 5,4 - Malasaña,Calle Manuela Malasaña nº 5,7314.298725


# Resultado final 

In [51]:
tabla_final_sorted = tabla_final.sort_values(by=['Place of interest', 'Distance'])
tabla_final_sorted

,Place of interest,Type of Place,Place Address,BiciMAD Station,Station Location,Distance
263,Campo de Golf del Centro Nacional de Golf,Principales espacios de deporte,CALLE ARROYO DEL MONTE 5,261 - Facultad Derecho,Avenida Complutense nº 23,3887.705045
262,Campo de Golf del Centro Nacional de Golf,Principales espacios de deporte,CALLE ARROYO DEL MONTE 5,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,4233.180269
241,Campo de Golf del Centro Nacional de Golf,Principales espacios de deporte,CALLE ARROYO DEL MONTE 5,239 - Francos Rodríguez,Calle Francos Rodríguez nº 62,4417.980657
239,Campo de Golf del Centro Nacional de Golf,Principales espacios de deporte,CALLE ARROYO DEL MONTE 5,237 - Pablo Iglesias,Calle Francisco Balseiro nº 1,4700.920904
260,Campo de Golf del Centro Nacional de Golf,Principales espacios de deporte,CALLE ARROYO DEL MONTE 5,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,4739.575947
...,...,...,...,...,...,...
158924,WiZink Center,Principales espacios de deporte,CALLE JORGE JUAN 99,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,6533.565460
158923,WiZink Center,Principales espacios de deporte,CALLE JORGE JUAN 99,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,6687.479181
158883,WiZink Center,Principales espacios de deporte,CALLE JORGE JUAN 99,217 - Pedro Rico,Calle Pedro Rico nº 4,6689.545344
158926,WiZink Center,Principales espacios de deporte,CALLE JORGE JUAN 99,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,6759.459954


In [52]:
grouped = tabla_final_sorted.groupby('Place of interest')

In [53]:
resultado_final = grouped.head(1)

In [54]:
resultado_final

,Place of interest,Type of Place,Place Address,BiciMAD Station,Station Location,Distance
263,Campo de Golf del Centro Nacional de Golf,Principales espacios de deporte,CALLE ARROYO DEL MONTE 5,261 - Facultad Derecho,Avenida Complutense nº 23,3887.705045
496,Campo de Golf del Club de Golf Olivar de la Hi...,Principales espacios de deporte,CALLE VIA DE DUBLIN SIN NÚMERO,230 - Carlota O’Neill,Calle Carlota O'Neill nº 32,5235.404881
760,Centro Comercial Dreams - Palacio Municipal de...,Principales espacios de deporte,CALLE SILVANO 77,230 - Carlota O’Neill,Calle Carlota O'Neill nº 32,2591.237763
1529,Centro Deportivo Municipal María Jesús Rosa (...,Principales espacios de deporte,AVENIDA MONFORTE DE LEMOS 38,207 - Remonta,Calle Azucenas nº 1,2162.785163
1672,Centro Deportivo Municipal Alcántara,Principales espacios de deporte,CALLE ALCANTARA 26,89 - Conde Peñalver,Calle Ayala nº 102,189.091233
...,...,...,...,...,...,...
157871,Parque Deportivo Puerta de Hierro,Principales espacios de deporte,CARRETERA DE LA CORUÑA kilómetro 7,261 - Facultad Derecho,Avenida Complutense nº 23,2641.898592
158135,Parque Lineal UZI 0.06 Arroyofresno,Principales espacios de deporte,CALLE MARIA DE MAEZTU 1,261 - Facultad Derecho,Avenida Complutense nº 23,4032.665528
158362,Pista de Patinaje Paco de Lucía,Principales espacios de deporte,AUDITORIO PACO DE LUCIA,224 - Caramuel,Calle Caramuel nº 51,4779.660155
158609,Pista de petanca Centro de Mayores La Vaguada,Principales espacios de deporte,AVENIDA MONFORTE DE LEMOS 36,207 - Remonta,Calle Azucenas nº 1,2159.005542


In [55]:
resultado_final.to_csv('../data/main_challenge_csv.csv', index=False)